In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as colors
import matplotlib.path as mpath
import scipy.io
import matplotlib.gridspec as gridspec

# functions from gasex module
from gasex.phys import vpress_sw
from gasex.fugacity import fugacity_factor

In [2]:
# helper function
def generateinputs(tests, windproduct):
    parameterizations = ["W14", "L13"]
    #windproducts = ["ncep"]    
    inputs = [[p,w, t] for p in parameterizations for w in windproduct for t in tests]
    print(f"inputs: {inputs}")
    return inputs

# figure 1

In [3]:
outputpath = 'datasets'
t = pq.read_table(f"{outputpath}/n2opredictions.parquet")
data = t.to_pandas()

data["msl"] = data[["msl_era5", "msl_ncep"]].mean(axis = 1)
data["U10"] = data[["U10_era5","U10_ncep"]].mean(axis = 1)
data["SI"] = data.SI_era5

# constants for converting umol/m2/day to Tg N2O-N/year
Ngpermol = 14.0067
Tgperg = 1e12
umolpermol = 1e6

# properties needed for area-time integration
areas = pd.DataFrame([["STZ",2.26e7],["SAZ",1.94e7],["PFZ",1.43e7],
                           ["ASZ",1.28e7],["SIZ",1.72e7],["TOTAL",8.64e7]],
                columns = ["zone","Area_km2"]).set_index("zone")

areas["m2"] = areas.Area_km2*1e6 # convert to m2 because fluxes are in umol/m2/day

daysinmonth = pd.DataFrame([[1.0, 31],
                            [2.0,28],
                            [3.0,31],
                            [4.0,30],
                            [5.0,31],
                            [6.0,30],
                            [7.0, 31],
                            [8.0,31],
                            [9.0,30],
                            [10.0,31],
                            [11.0,30],
                            [12.0,31],
                           ],
                           columns = ["month","daysinmonth"]).set_index("month")
Ft = np.load(f"{outputpath}/fluxtests/observedcombined_mean.npy")
Ftstdev = np.load(f"{outputpath}/fluxtests/observedcombined_stdev.npy")

data["Ft"] = Ft
data["Ftstdev"] = Ftstdev

condition = "observed"
inputs = generateinputs([condition], ["ncep", "era5"])
labels = []

for test in inputs:
    p = test[0]
    t = test[2]
    w = test[1]
    
    Ft = np.load(f"{outputpath}/fluxtests/{t}{w}{p}_mean.npy")
    Ftstdev = np.load(f"{outputpath}/fluxtests/{test[2]}{test[1]}{test[0]}_stdev.npy")
    
    meanlabel = f"{t}{w}{p}"
    stdevlabel = f"{t}{w}{p}_stdev"

    print(meanlabel, stdevlabel)
    labels.append(meanlabel)

    data[meanlabel] = Ft
    data[stdevlabel] = Ftstdev

data["Ft"] = np.load(f'{outputpath}/fluxtests/{condition}combined_mean.npy')
data["pN2O_uncertainty"] = np.load(f'{outputpath}/fluxtests/{condition}combined_stdev.npy')
data["wind_uncertainty"] = data[labels].std(axis=1)
data["Ftstdev"] = np.sqrt(data["pN2O_uncertainty"]**2 + data["wind_uncertainty"]**2)

tests = ["baseline", "noice", "1atm", "medmsl", "meanmsl", "medK", "meanK", "medN2O",
    "meanN2O", # for whatever reason, this one throws an error
    "WINDS", "COMBINED", "CYCLONES"
    ]
for condition in tests:
    inputs = generateinputs([condition], ["ncep", "era5"])
    labels = []
    
    for test in inputs:
        p = test[0]
        t = test[2]
        w = test[1]
        
        Ft = np.load(f"{outputpath}/fluxtests/{t}{w}{p}_mean.npy")
        Ftstdev = np.load(f"{outputpath}/fluxtests/{test[2]}{test[1]}{test[0]}_stdev.npy")
        
        meanlabel = f"{t}{w}{p}"
        stdevlabel = f"{t}{w}{p}_stdev"
    
        print(meanlabel, stdevlabel)
        labels.append(meanlabel)
    
        data[meanlabel] = Ft
        data[stdevlabel] = Ftstdev
    
    data[f"Ft{condition}"] = np.load(f'{outputpath}/fluxtests/{condition}combined_mean.npy')
    pN2O_uncertainty_condition = np.load(f'{outputpath}/fluxtests/{condition}combined_stdev.npy')
    wind_uncertainty_condition = data[labels].std(axis=1)
    data[f"Ft{condition}stdev"] = np.sqrt(pN2O_uncertainty_condition**2 + wind_uncertainty_condition**2)

data = data.copy() # avoid fragmenting dataframe

ph2ov = vpress_sw(data.SP,data.pt) # atm
f  = fugacity_factor(data.pt, gas='N2O',slp=data.msl)
data["pN2Oatm"] = data.XN2Oa*1e9 * f * (data.msl - ph2ov)
data["pN2Oatm_1atm"] = data.XN2Oa*1e9 * f * (1 - ph2ov)
data["DpN2O_pred"] = data.pN2O_pred -data["pN2Oatm"]
data["DpN2O_pred2"] = data.pN2O_pred -data["pN2Oatm_1atm"]
data["DpN2O_pred3"] = np.median(data.pN2O_pred) - data["pN2Oatm"]

data.to_csv('datasets/fig1.csv')

inputs: [['W14', 'ncep', 'observed'], ['W14', 'era5', 'observed'], ['L13', 'ncep', 'observed'], ['L13', 'era5', 'observed']]
observedncepW14 observedncepW14_stdev
observedera5W14 observedera5W14_stdev
observedncepL13 observedncepL13_stdev
observedera5L13 observedera5L13_stdev
inputs: [['W14', 'ncep', 'baseline'], ['W14', 'era5', 'baseline'], ['L13', 'ncep', 'baseline'], ['L13', 'era5', 'baseline']]
baselinencepW14 baselinencepW14_stdev
baselineera5W14 baselineera5W14_stdev
baselinencepL13 baselinencepL13_stdev
baselineera5L13 baselineera5L13_stdev
inputs: [['W14', 'ncep', 'noice'], ['W14', 'era5', 'noice'], ['L13', 'ncep', 'noice'], ['L13', 'era5', 'noice']]
noicencepW14 noicencepW14_stdev
noiceera5W14 noiceera5W14_stdev
noicencepL13 noicencepL13_stdev
noiceera5L13 noiceera5L13_stdev
inputs: [['W14', 'ncep', '1atm'], ['W14', 'era5', '1atm'], ['L13', 'ncep', '1atm'], ['L13', 'era5', '1atm']]
1atmncepW14 1atmncepW14_stdev
1atmera5W14 1atmera5W14_stdev
1atmncepL13 1atmncepL13_stdev
1atmer

/tmp/ipykernel_157309/2299199842.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[meanlabel] = Ft
/tmp/ipykernel_157309/2299199842.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[stdevlabel] = Ftstdev
/tmp/ipykernel_157309/2299199842.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy(

WINDSncepW14 WINDSncepW14_stdev
WINDSera5W14 WINDSera5W14_stdev
WINDSncepL13 WINDSncepL13_stdev
WINDSera5L13 WINDSera5L13_stdev
inputs: [['W14', 'ncep', 'COMBINED'], ['W14', 'era5', 'COMBINED'], ['L13', 'ncep', 'COMBINED'], ['L13', 'era5', 'COMBINED']]


/tmp/ipykernel_157309/2299199842.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[meanlabel] = Ft
/tmp/ipykernel_157309/2299199842.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[stdevlabel] = Ftstdev
/tmp/ipykernel_157309/2299199842.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy(

COMBINEDncepW14 COMBINEDncepW14_stdev
COMBINEDera5W14 COMBINEDera5W14_stdev
COMBINEDncepL13 COMBINEDncepL13_stdev
COMBINEDera5L13 COMBINEDera5L13_stdev
inputs: [['W14', 'ncep', 'CYCLONES'], ['W14', 'era5', 'CYCLONES'], ['L13', 'ncep', 'CYCLONES'], ['L13', 'era5', 'CYCLONES']]
CYCLONESncepW14 CYCLONESncepW14_stdev


/tmp/ipykernel_157309/2299199842.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[meanlabel] = Ft
/tmp/ipykernel_157309/2299199842.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[stdevlabel] = Ftstdev
/tmp/ipykernel_157309/2299199842.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy(

CYCLONESera5W14 CYCLONESera5W14_stdev
CYCLONESncepL13 CYCLONESncepL13_stdev
CYCLONESera5L13 CYCLONESera5L13_stdev


/tmp/ipykernel_157309/2299199842.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[meanlabel] = Ft
/tmp/ipykernel_157309/2299199842.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[stdevlabel] = Ftstdev
/tmp/ipykernel_157309/2299199842.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy(

# figure 2

In [4]:
outputpath = 'datasets'
t = pq.read_table(f"{outputpath}/n2opredictions.parquet")
surface = t.to_pandas()

inputs = generateinputs(["observed"], ["ncep", "era5"])

for test in inputs:
    p = test[0]
    t = test[2]
    w = test[1]
    
    Ft = np.load(f"{outputpath}/fluxtests/{t}{w}{p}_mean.npy")
    Ftstdev = np.load(f"{outputpath}/fluxtests/{test[2]}{test[1]}{test[0]}_stdev.npy")
    
    meanlabel = f"{t}{w}{p}"
    stdevlabel = f"{t}{w}{p}_stdev"

    surface[meanlabel] = Ft*surface.m2*surface.daysinmonth/surface.msl_era5count*(Ngpermol*2)/Tgperg/umolpermol
    surface[stdevlabel] = Ftstdev*surface.m2*surface.daysinmonth/surface.msl_era5count*(Ngpermol*2)/Tgperg/umolpermol

surface["combined_mean"] = np.load(f'{outputpath}/fluxtests/{"observed"}combined_mean.npy')*surface.m2*surface.daysinmonth/surface.msl_era5count*(Ngpermol*2)/Tgperg/umolpermol
surface["combined_uncertainty"] = np.load(f'{outputpath}/fluxtests/{"observed"}combined_stdev.npy')*surface.m2*surface.daysinmonth/surface.msl_era5count*(Ngpermol*2)/Tgperg/umolpermol

data_sorted = surface.sort_values(f"msl_era5")

mean_cumsum = np.cumsum(np.array(data_sorted["combined_mean"]))
mslarray = np.array(data_sorted[f"msl_era5"])

print(mean_cumsum[-1])
cumsums = np.empty((len(data_sorted),len(inputs)))

for count, test in enumerate(inputs):
    p = test[0]
    t = test[2]
    w = test[1]

    cumsum = np.cumsum(np.array(data_sorted[f"{t}{w}{p}"]))
    cumsums[:,count] = cumsum

    uncertainties = np.array(data_sorted[f"{t}{w}{p}_stdev"])
    uncertaintiessquared = uncertainties**2
    uncertaintiescumulative = np.cumsum(uncertaintiessquared)
    uncertainties = np.sqrt(uncertaintiescumulative)

wind_uncertainty = np.std(cumsums, axis = 1)

montecarlo_uncertainty = np.cumsum(np.array(data_sorted["combined_uncertainty"])**2)

bootstrap_uncertainty = 0.038*mean_cumsum#bootstrap_uncertainties[260]/np.mean(bootstrap_sums[260,:])*mean_cumsum

total_uncertainty = np.sqrt(
    wind_uncertainty**2 + 
    montecarlo_uncertainty + 
    bootstrap_uncertainty**2
)

data_sorted["mslarray"] = mslarray
data_sorted["mean_cumsum"] = mean_cumsum
data_sorted["wind_uncertainty"] = wind_uncertainty
data_sorted["montecarlo_uncertainty"] = montecarlo_uncertainty
data_sorted["bootstrap_uncertainty"] = bootstrap_uncertainty
data_sorted["total_uncertainty"] = total_uncertainty
data_sorted.to_csv("datasets/fig2a.csv")

inputs: [['W14', 'ncep', 'observed'], ['W14', 'era5', 'observed'], ['L13', 'ncep', 'observed'], ['L13', 'era5', 'observed']]
1.5954010434742338


In [5]:
outputpath = 'datasets'
t = pq.read_table(f"{outputpath}/n2opredictions.parquet")
surface = t.to_pandas()

inputs = generateinputs(["1atm"], ["ncep", "era5"])

for test in inputs:
    p = test[0]
    t = test[2]
    w = test[1]
    
    Ft = np.load(f"{outputpath}/fluxtests/{t}{w}{p}_mean.npy")
    Ftstdev = np.load(f"{outputpath}/fluxtests/{test[2]}{test[1]}{test[0]}_stdev.npy")
    
    meanlabel = f"{t}{w}{p}"
    stdevlabel = f"{t}{w}{p}_stdev"

    surface[meanlabel] = Ft*surface.m2*surface.daysinmonth/surface.msl_era5count*(Ngpermol*2)/Tgperg/umolpermol
    surface[stdevlabel] = Ftstdev*surface.m2*surface.daysinmonth/surface.msl_era5count*(Ngpermol*2)/Tgperg/umolpermol

surface["combined_mean"] = np.load(f'{outputpath}/fluxtests/{"1atm"}combined_mean.npy')*surface.m2*surface.daysinmonth/surface.msl_era5count*(Ngpermol*2)/Tgperg/umolpermol
surface["combined_uncertainty"] = np.load(f'{outputpath}/fluxtests/{"1atm"}combined_stdev.npy')*surface.m2*surface.daysinmonth/surface.msl_era5count*(Ngpermol*2)/Tgperg/umolpermol

data_sorted = surface.sort_values(f"msl_era5")

mean_cumsum = np.cumsum(np.array(data_sorted["combined_mean"]))
mslarray = np.array(data_sorted[f"msl_era5"])

print(mean_cumsum[-1])
cumsums = np.empty((len(data_sorted),len(inputs)))

for count, test in enumerate(inputs):
    p = test[0]
    t = test[2]
    w = test[1]

    cumsum = np.cumsum(np.array(data_sorted[f"{t}{w}{p}"]))
    cumsums[:,count] = cumsum

    uncertainties = np.array(data_sorted[f"{t}{w}{p}_stdev"])
    uncertaintiessquared = uncertainties**2
    uncertaintiescumulative = np.cumsum(uncertaintiessquared)
    uncertainties = np.sqrt(uncertaintiescumulative)

wind_uncertainty = np.std(cumsums, axis = 1)

montecarlo_uncertainty = np.cumsum(np.array(data_sorted["combined_uncertainty"])**2)

bootstrap_uncertainty = 0.038*mean_cumsum#bootstrap_uncertainties[260]/np.mean(bootstrap_sums[260,:])*mean_cumsum

total_uncertainty = np.sqrt(
    wind_uncertainty**2 + 
    montecarlo_uncertainty + 
    bootstrap_uncertainty**2
)

data_sorted["mslarray"] = mslarray
data_sorted["mean_cumsum"] = mean_cumsum
data_sorted["wind_uncertainty"] = wind_uncertainty
data_sorted["montecarlo_uncertainty"] = montecarlo_uncertainty
data_sorted["bootstrap_uncertainty"] = bootstrap_uncertainty
data_sorted["total_uncertainty"] = total_uncertainty
data_sorted.to_csv("datasets/fig2b.csv")

inputs: [['W14', 'ncep', '1atm'], ['W14', 'era5', '1atm'], ['L13', 'ncep', '1atm'], ['L13', 'era5', '1atm']]
0.8929258341553027


# figure 3

In [7]:
conditions = ["observed", "medK", "1atm", "baseline"]
totals = []
mids = []
data_sorted = surface.sort_values(f"msl_era5")

for i, condition in enumerate(conditions):
    inputs = generateinputs([condition], ["ncep", "era5"])
    
    for test in inputs:
        p = test[0]
        t = test[2]
        w = test[1]
        
        Ft = np.load(f"{outputpath}/fluxtests/{t}{w}{p}_mean.npy")
        Ftstdev = np.load(f"{outputpath}/fluxtests/{test[2]}{test[1]}{test[0]}_stdev.npy")
        
        meanlabel = f"{t}{w}{p}"
        stdevlabel = f"{t}{w}{p}_stdev"
    
        data_sorted[meanlabel] = Ft*surface.m2*surface.daysinmonth/surface.msl_era5count*(Ngpermol*2)/Tgperg/umolpermol
        data_sorted[stdevlabel] = Ftstdev*surface.m2*surface.daysinmonth/surface.msl_era5count*(Ngpermol*2)/Tgperg/umolpermol
    
    data_sorted["combined_mean"] = np.load(f'{outputpath}/fluxtests/{condition}combined_mean.npy')*surface.m2*surface.daysinmonth/surface.msl_era5count*(Ngpermol*2)/Tgperg/umolpermol
    data_sorted["combined_uncertainty"] = np.load(f'{outputpath}/fluxtests/{condition}combined_stdev.npy')*surface.m2*surface.daysinmonth/surface.msl_era5count*(Ngpermol*2)/Tgperg/umolpermol
    
    mean_cumsum = np.cumsum(np.array(data_sorted["combined_mean"]))
    data_sorted[f"Ft_cumulative{condition}"] = mean_cumsum
    
    mslarray = np.array(data_sorted[f"msl_era5"])

    totals.append(mean_cumsum[-1])
    msl50 = np.median(mslarray)
    mslmask = data_sorted[f"msl_era5"]<=msl50
    mids.append(mean_cumsum[mslmask][-1])
    
    cumsums = np.empty((len(data_sorted),len(inputs)))
    
    for count, test in enumerate(inputs):
        p = test[0]
        t = test[2]
        w = test[1]
    
        cumsum = np.cumsum(np.array(data_sorted[f"{t}{w}{p}"]))
        cumsums[:,count] = cumsum
    
    wind_uncertainty = np.std(cumsums, axis = 1)
    
    montecarlo_uncertainty = np.cumsum(np.array(data_sorted["combined_uncertainty"])**2)
    
    bootstrap_uncertainty = 0.0385*mean_cumsum#bootstrap_uncertainties[260]/np.mean(bootstrap_sums[260,:])*mean_cumsum
    
    total_uncertainty = np.sqrt(
        wind_uncertainty**2 + 
        montecarlo_uncertainty + 
        bootstrap_uncertainty**2
    )
    print(total_uncertainty[-1])

data_sorted.to_csv("datasets/fig3.csv")

inputs: [['W14', 'ncep', 'observed'], ['W14', 'era5', 'observed'], ['L13', 'ncep', 'observed'], ['L13', 'era5', 'observed']]
0.3211880859077775
inputs: [['W14', 'ncep', 'medK'], ['W14', 'era5', 'medK'], ['L13', 'ncep', 'medK'], ['L13', 'era5', 'medK']]
0.13933789074149894
inputs: [['W14', 'ncep', '1atm'], ['W14', 'era5', '1atm'], ['L13', 'ncep', '1atm'], ['L13', 'era5', '1atm']]
0.33390530737762725
inputs: [['W14', 'ncep', 'baseline'], ['W14', 'era5', 'baseline'], ['L13', 'ncep', 'baseline'], ['L13', 'era5', 'baseline']]
0.10866104187122427


# figure 4

In [11]:
conditions = ["1atm", "medK", "medN2O", "observed", "noice", "WINDS", "COMBINED"]
monthlycolumns = ['zone', 'month']
seasonalcolumns = ['zone', 'season']
annualcolumns = ['month']
zonalcolumns = ['zone']

wind_uncertainties_monthly = []
wind_uncertainties_annual = []
wind_uncertainties_seasonal = []
wind_uncertainties_zonal = []

annual = data.copy()
seasonarray = np.copy(np.array(data.zone))
seasonarray[np.isin(annual.month, [4,5,6])] = 'peak'
seasonarray[np.isin(annual.month, [10,11,12])] = 'trough'
seasonarray[np.isin(annual.month, [1,2,3,7,8,9])] = 'none'
annual['season'] = seasonarray
annual["doy"] = np.array([pd.Timestamp(d).dayofyear for d in annual.loc[:,'JULD'].values])

for i, condition in enumerate(conditions):
    inputs = generateinputs([condition], ["ncep", "era5"])
    monthlylabels = ['zone', 'month']
    seasonallabels = ['zone', 'season']
    annuallabels = ['month']
    zonallabels = ['zone']
    
    for test in inputs:
        p = test[0]
        w = test[1]
        t = test[2]
        
        Ft = np.load(f"{outputpath}/fluxtests/{t}{w}{p}_mean.npy")
        
        meanlabel = f"{t}{w}{p}"
        for lst in [monthlylabels, seasonallabels, annuallabels, zonallabels]:
            lst.append(meanlabel)

        annual[meanlabel] = Ft*annual.m2*annual.daysinmonth/annual.msl_era5count*(Ngpermol*2)/Tgperg/umolpermol

    wind_uncertainty = np.array(annual[monthlylabels].groupby(['zone','month']).sum().std(axis = 1))
    wind_uncertainties_monthly.append(wind_uncertainty)

    wind_uncertainty = np.array(annual[annuallabels].groupby('month').sum().std(axis = 1))
    wind_uncertainties_annual.append(wind_uncertainty)

    wind_uncertainty = np.array(annual[seasonallabels].groupby(['zone','season']).sum().std(axis = 1))*4
    wind_uncertainties_seasonal.append(wind_uncertainty)

    wind_uncertainty = np.array(annual[zonallabels].groupby('zone').sum().std(axis = 1))
    wind_uncertainties_zonal.append(wind_uncertainty)
    
    annual[f"mean_flux{condition}"] = np.load(f'{outputpath}/fluxtests/{condition}combined_mean.npy')*annual.m2*annual.daysinmonth/annual.msl_era5count*(Ngpermol*2)/Tgperg/umolpermol
    annual[f"montecarlo_uncertainty{condition}"] = np.load(f'{outputpath}/fluxtests/{condition}combined_stdev.npy')*annual.m2*annual.daysinmonth/annual.msl_era5count*(Ngpermol*2)/Tgperg/umolpermol
    annual[f"montecarlo_uncertainty_squared{condition}"] = annual[f"montecarlo_uncertainty{condition}"]**2

    for lst in [monthlycolumns, seasonalcolumns, annualcolumns, zonalcolumns]:
        lst.append(f"mean_flux{condition}")
        lst.append(f"montecarlo_uncertainty_squared{condition}")

grouped = annual[monthlycolumns].groupby(['zone','month']).sum()

for i, condition in enumerate(conditions):
    grouped[f"wind_uncertainty{condition}"] = wind_uncertainties_monthly[i]
    grouped[f"bootstrap_uncertainty{condition}"] = 0.0385*grouped[f"mean_flux{condition}"]
    grouped[f"total_uncertainty{condition}"] = np.sqrt(grouped[f"montecarlo_uncertainty_squared{condition}"] + 
                                                       grouped[f"wind_uncertainty{condition}"]**2 +
                                                       grouped[f"bootstrap_uncertainty{condition}"]**2)

overall = annual[annualcolumns].groupby(['month']).sum()

for i, condition in enumerate(conditions):
    overall[f"wind_uncertainty{condition}"] = wind_uncertainties_annual[i]
    overall[f"bootstrap_uncertainty{condition}"] = 0.0385*overall[f"mean_flux{condition}"]
    overall[f"total_uncertainty{condition}"] = np.sqrt(overall[f"montecarlo_uncertainty_squared{condition}"] + 
                                                       overall[f"wind_uncertainty{condition}"]**2 +
                                                       overall[f"bootstrap_uncertainty{condition}"]**2)

overall.to_csv("datasets/fig4.csv")
grouped.to_csv('datasets/fig5column2.csv')

inputs: [['W14', 'ncep', '1atm'], ['W14', 'era5', '1atm'], ['L13', 'ncep', '1atm'], ['L13', 'era5', '1atm']]
inputs: [['W14', 'ncep', 'medK'], ['W14', 'era5', 'medK'], ['L13', 'ncep', 'medK'], ['L13', 'era5', 'medK']]
inputs: [['W14', 'ncep', 'medN2O'], ['W14', 'era5', 'medN2O'], ['L13', 'ncep', 'medN2O'], ['L13', 'era5', 'medN2O']]
inputs: [['W14', 'ncep', 'observed'], ['W14', 'era5', 'observed'], ['L13', 'ncep', 'observed'], ['L13', 'era5', 'observed']]
inputs: [['W14', 'ncep', 'noice'], ['W14', 'era5', 'noice'], ['L13', 'ncep', 'noice'], ['L13', 'era5', 'noice']]
inputs: [['W14', 'ncep', 'WINDS'], ['W14', 'era5', 'WINDS'], ['L13', 'ncep', 'WINDS'], ['L13', 'era5', 'WINDS']]
inputs: [['W14', 'ncep', 'COMBINED'], ['W14', 'era5', 'COMBINED'], ['L13', 'ncep', 'COMBINED'], ['L13', 'era5', 'COMBINED']]


# figure 5

In [10]:
monthlyzonalmeans = data[["zone", "month", "DpN2O_pred", "DpN2O_pred2", "DpN2O_pred3"]].groupby(["zone", "month"]).mean()
monthlyzonalstds = data[["zone", "month", "DpN2O_pred", "DpN2O_pred2", "DpN2O_pred3"]].groupby(["zone", "month"]).std()
monthlyzonalcounts = data[["zone", "month", "DpN2O_pred", "DpN2O_pred2", "DpN2O_pred3"]].groupby(["zone", "month"]).count()
monthlyzonalmeans = monthlyzonalmeans.join(monthlyzonalstds, rsuffix = "_std")
monthlyzonalmeans = monthlyzonalmeans.join(monthlyzonalcounts, rsuffix = "_count")
monthlyzonalmeans['DpN2O_pred_sem'] = monthlyzonalmeans['DpN2O_pred_std']/np.sqrt(monthlyzonalmeans.DpN2O_pred_count)
monthlyzonalmeans['DpN2O_pred2_sem'] = monthlyzonalmeans['DpN2O_pred2_std']/np.sqrt(monthlyzonalmeans.DpN2O_pred2_count)
monthlyzonalmeans['DpN2O_pred3_sem'] = monthlyzonalmeans['DpN2O_pred3_std']/np.sqrt(monthlyzonalmeans.DpN2O_pred3_count)

monthlyzonalmeans.to_csv('datasets/fig5.csv')

grouped = annual[monthlycolumns].groupby(['zone','month']).sum()

for i, condition in enumerate(conditions):
    grouped[f"wind_uncertainty{condition}"] = wind_uncertainties_monthly[i]
    grouped[f"bootstrap_uncertainty{condition}"] = 0.0385*grouped[f"mean_flux{condition}"]
    grouped[f"total_uncertainty{condition}"] = np.sqrt(grouped[f"montecarlo_uncertainty_squared{condition}"] + 
                                                       grouped[f"wind_uncertainty{condition}"]**2 +
                                                       grouped[f"bootstrap_uncertainty{condition}"]**2)